This notebook provides code to qury the OpenStreetMap API to identify the number of buildings and the total building area within a 1/2 mile radius of a fixed point. For more information please visit https://www.transitorienteddiscoveries.com/blog.

In [1]:
import requests
from shapely.geometry import Point, Polygon, shape
import geopandas as gpd

# Define constants
latitude = 37.8931
longitude = -122.12463
radius_miles = 0.5
radius_meters = radius_miles * 1609.34  # convert miles to meters

# Function to query Overpass API
def query_overpass(lat, lon, radius):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      way["building"](around:{radius},{lat},{lon});
    );
    out body;
    >;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()
    return data

# Function to calculate the area occupied by buildings
def calculate_building_area(data):
    # Convert response to GeoDataFrame
    elements = data['elements']
    ways = [el for el in elements if el['type'] == 'way']
    coords = {el['id']: (el['lon'], el['lat']) for el in elements if el['type'] == 'node'}

    polygons = []
    for way in ways:
        polygon_coords = [(coords[node_id][0], coords[node_id][1]) for node_id in way['nodes'] if node_id in coords]
        if polygon_coords and polygon_coords[0] == polygon_coords[-1]:  # check if it's a closed polygon
            polygons.append(Polygon(polygon_coords))

    gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")
    gdf = gdf.to_crs(epsg=32610)  # Convert to a metric CRS (UTM zone 10N for the Bay Area)

    total_area_m2 = gdf.area.sum()
    total_area_ft2 = total_area_m2 * 10.7639  # Convert square meters to square feet
    return len(polygons), total_area_ft2

# Perform the query and calculate the area
data = query_overpass(latitude, longitude, radius_meters)
num_buildings, total_area_ft2 = calculate_building_area(data)

# Print the results
print(f"Number of buildings within {radius_miles} mile: {num_buildings}")
print(f"Total land area occupied by buildings: {total_area_ft2:.2f} square feet")


Number of buildings within 0.5 mile: 796
Total land area occupied by buildings: 3207287.84 square feet
